In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd '/content/gdrive/MyDrive/LDS9_K265_TranHoangBach/Week_2/data_day_4'

Mounted at /content/gdrive
/content/gdrive/MyDrive/LDS9_K265_TranHoangBach/Week_2/data_day_4


In [22]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sb
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
import time

In [4]:
sc = SparkContext(master="local", appName="New Spark Context")
spark = SparkSession(sc)

In [5]:
df = spark.read.csv(["AA_data"], header=True, inferSchema=True)
df.show(5)

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2014|            5|                HNL|                          519|
|       01/01/2014|            7|                OGG|                          505|
|       01/01/2014|           35|                SLC|                          174|
|       01/01/2014|           43|                DTW|                          153|
|       01/01/2014|           52|                PIT|                          137|
+-----------------+-------------+-------------------+-----------------------------+
only showing top 5 rows



In [6]:
df.count()

583718

In [7]:
df.printSchema()

root
 |-- Date (MM/DD/YYYY): string (nullable = true)
 |-- Flight Number: integer (nullable = true)
 |-- Destination Airport: string (nullable = true)
 |-- Actual elapsed time (Minutes): integer (nullable = true)



In [8]:
df.orderBy(*[df['Date (MM/DD/YYYY)'].desc()]).show(5)

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       12/31/2017|            5|                HNL|                          615|
|       12/31/2017|          103|                SJC|                          334|
|       12/31/2017|            7|                OGG|                          525|
|       12/31/2017|           43|                DTW|                          217|
|       12/31/2017|           90|                ORD|                          152|
+-----------------+-------------+-------------------+-----------------------------+
only showing top 5 rows



In [9]:
print('Number of Null values')
for col_name in df.columns:
    print('\t -', col_name, ": ", df.where(df[col_name].isNull()).count())

Number of Null values
	 - Date (MM/DD/YYYY) :  0
	 - Flight Number :  0
	 - Destination Airport :  0
	 - Actual elapsed time (Minutes) :  0


In [10]:
df = df.drop_duplicates()

In [11]:
df.count()

583718

In [13]:
df = df.withColumnRenamed('Destination Airport', 'airport')
df = df.withColumnRenamed('Date (MM/DD/YYYY)', 'date')
df = df.withColumnRenamed('Flight Number', 'flight_num')
df = df.withColumnRenamed('Actual elapsed time (Minutes)', 'actual_time')
df.show(5)

+----------+----------+-------+-----------+
|      date|flight_num|airport|actual_time|
+----------+----------+-------+-----------+
|01/01/2014|      2372|    MSY|         74|
|01/01/2014|      2419|    LAX|        182|
|01/03/2014|      1646|    TUL|         52|
|01/03/2014|      2201|    TPA|        141|
|01/03/2014|      2248|    SEA|        261|
+----------+----------+-------+-----------+
only showing top 5 rows



In [14]:
def convert_datetime(x):
    try:
        return datetime.strptime(x, '%m/%d/%Y')
    except Exception:
        return None
# func = udf(lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
func = udf(lambda x: convert_datetime(x), DateType())
df = df.withColumn('date', func(df.date))

In [15]:
df.show(5)

+----------+----------+-------+-----------+
|      date|flight_num|airport|actual_time|
+----------+----------+-------+-----------+
|2014-01-01|      2372|    MSY|         74|
|2014-01-01|      2419|    LAX|        182|
|2014-01-03|      1646|    TUL|         52|
|2014-01-03|      2201|    TPA|        141|
|2014-01-03|      2248|    SEA|        261|
+----------+----------+-------+-----------+
only showing top 5 rows



In [16]:
df.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

In [17]:
df_new = spark.read.parquet('AA_DFW_ALL.parquet')
df_new.show(5)

+----------+----------+-------+-----------+
|      date|flight_num|airport|actual_time|
+----------+----------+-------+-----------+
|2014-01-01|      1431|    SEA|        259|
|2014-01-02|        60|    MIA|        167|
|2014-01-02|       294|    DTW|        151|
|2014-01-02|      1277|    BWI|        142|
|2014-01-03|      1467|    ABQ|        102|
+----------+----------+-------+-----------+
only showing top 5 rows



In [23]:
print('Is df is cached? {}'.format(df_new.is_cached))
# df.cache()
# print('Is df is cached? {}'.format(df.is_cached))
# df.unpersist()
# print('Is df is cached? {}'.format(df.is_cached))

Is df is cached? False


In [19]:
df_new.createOrReplaceTempView('flights')

In [21]:
query = 'SELECT AVG(actual_time) from flights'
spark.sql(query).show()

+------------------+
|  avg(actual_time)|
+------------------+
|147.59399915712726|
+------------------+



In [24]:
def count_w_timer(df):
    print('Cache Status: {}'.format(df.is_cached))
    tic = time.time()
    print('Counting result: {} rows, took {:.2f}'.format(df.count(), time.time()-tic))

count_w_timer(df_new)

Cache Status: False
Counting result: 583718 rows, took 4.43


In [26]:
df_new.cache()
count_w_timer(df_new)

Cache Status: True
Counting result: 583718 rows, took 0.11


In [27]:
df_new.unpersist()
print('Is df is cached? {}'.format(df_new.is_cached))

Is df is cached? False


In [28]:
app_name = spark.conf.get('spark.app.name')
driver_tcp_port = spark.conf.get('spark.driver.port')
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

print('Name: %s' %app_name)
print('Driver TCP port: %s' %driver_tcp_port)
print('Number of partitions: %s' %num_partitions)

Name: New Spark Context
Driver TCP port: 34567
Number of partitions: 200


In [31]:
before = df_new.rdd.getNumPartitions()
spark.conf.set('spark.sql.shuffle.partitions', 500)
df_new = spark.read.parquet('AA_DFW_ALL.parquet').distinct()

print('Partition count before change: %d' %before)
print('Partition count after  change: %d' %df_new.rdd.getNumPartitions())

Partition count before change: 7
Partition count after  change: 500
